# 2. Régression linéaire

In [128]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

In [129]:
df = pd.read_csv("data/data.csv", header=0)

In [130]:
# On recalcule les données de l'étape précédente

col_academic_performance = "AcademicPerformanceLastorCurrentAcademicYear"
col_screen_time = "ScreentimeHoursDayMobileTablet"
col_outdoor_time = "OutdoorplayingHoursDay"
col_bmi = "BMI"
academic_performance = (df[col_academic_performance] - df[col_academic_performance].mean()) / df[col_academic_performance].std()
screen_time = (df[col_screen_time] - df[col_screen_time].mean()) / df[col_screen_time].std()
outdoor_time = (df[col_outdoor_time] - df[col_outdoor_time].mean()) / df[col_outdoor_time].std()
bmi = (df[col_bmi] - df[col_bmi].mean()) / df[col_bmi].std()
top_percentile = 0.05
upper_threshold = academic_performance.quantile(1 - top_percentile)
lower_threshold = academic_performance.quantile(top_percentile)
mask = (academic_performance < upper_threshold) & (academic_performance > lower_threshold)
academic_performance = academic_performance[mask]
screen_time = screen_time[mask]
outdoor_time = outdoor_time[mask]
bmi = bmi[mask]
academic_performance_corrected = academic_performance.dropna()
screen_time_corrected = screen_time.dropna() ** (1/2)
outdoor_time_corrected = outdoor_time.dropna() ** (1/2)
bmi_corrected = bmi.dropna() ** (1/2)

In [131]:
import numpy as np

In [132]:
filter = pd.DataFrame({'x': screen_time_corrected, 'y':academic_performance_corrected})
filter = filter.dropna(subset=['x','y'])
valS = filter['x'].tolist()
valA = filter['y'].tolist()
avgS = np.average(valS)
avgA = np.average(valA)

## Calcul de la formule linéaire

On cherche un résultat sous la forme:
$$
A = a \cdot S + b
$$
Avec $A$ étant le score académique et $S$ le temps d'ecran.

Les formules pour trouver $a$ et $b$ sont les suivantes:
$$a = \frac{\sum_{i=1}^{n} (S_i - \overline{S})(A_i - \overline{A})}{\sum_{i=1}^{n} (S_i - \overline{A})^2}$$
et
$$b = \overline{A} - a \cdot \overline{S}$$

In [133]:
aup = 0
adown = 0
for i in range(len(valS)):
    aup += (valS[i]-avgS)*(valA[i]-avgA)
    adown += (valS[i]-avgS)**2

a = aup/adown
b = avgA - a*avgS
print(a, b)

-0.5685597261788912 0.2392578202068505


## Calcul du R²

pour calculer la pertinence de notre régression on obtiens le R² avec :
$$
R² = 1 - \frac {A_i - \overline{A}} {A_i - a \cdot S_i + b} 
$$

In [134]:
total = 0
res = 0
for i in range(len(valS)):
    distance = np.abs(avgA - a*valS[i]+b)
    total += (valA[i] - avgA)**2
    res += (valA[i] - distance)**2

RR = 1 - (total / res)

print(RR)

0.49551701541243864


Le R² va de 0 à 1, une valeur de $0.49$ indique que le temps d'écran permet en effet de prédire les résultats académiques, mais les résultats ne seront pas précis. On en déduit qu'il y a une corrélation modérée entre les deux paramètres.

## Calcul du R² pour d'autres paramètres

### Temps passé à joué dehors

In [135]:
filter = pd.DataFrame({'x': outdoor_time_corrected, 'y':academic_performance_corrected})
filter = filter.dropna(subset=['x','y'])
valO = filter['x'].tolist()
valA = filter['y'].tolist()
avgO = np.average(valO)
avgA = np.average(valA)

aup = 0
adown = 0
for i in range(len(valO)):
    aup += (valO[i]-avgO)*(valA[i]-avgA)
    adown += (valO[i]-avgO)**2
a = aup/adown
b = avgA - a*avgO

total = 0
res = 0
for i in range(len(valO)):
    distance = np.abs(avgA - a*valO[i]+b)
    total += (valA[i] - avgA)**2
    res += (valA[i] - distance)**2

RR = 1 - (total / res)

print(RR)

0.011983148374621111


$R² = 0.01$ , un R² aussi bas indique qu'il n'y a aucun lien entre nos deux paramètres. On ne peut donc pas utiliser le temps de jeu dehors pour prédire les résultats académiques.

### Body Mass Index

In [97]:
filter = pd.DataFrame({'x': bmi_corrected, 'y':academic_performance_corrected})
filter = filter.dropna(subset=['x','y'])
valB = filter['x'].tolist()
valA = filter['y'].tolist()
avgB = np.average(valB)
avgA = np.average(valA)

aup = 0
adown = 0
for i in range(len(valB)):
    aup += (valB[i]-avgB)*(valA[i]-avgA)
    adown += (valB[i]-avgB)**2
a = aup/adown
b = avgA - a*avgB

total = 0
res = 0
for i in range(len(valB)):
    distance = np.abs(avgA - a*valB[i]+b)
    total += (valA[i] - avgA)**2
    res += (valA[i] - distance)**2

RR = 1 - (total / res)

print(RR)

0.6395622624942826


$R² = 0.64$ , ces résultats nous montrent que l'indice de masse corporelle permet de mieux prédire les résultats académiques que le temps d'écran. L'amélioration n'est cependant pas majeure et la précision reste modérée.